In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import time
import openai
from dotenv import load_dotenv

from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup as bs

from method.ours.utils import create_driver, get_xpath
from method.ours.parse import parse_form
from method.ours.prompts import get_form_context
from method.ours.history import HistoryTable
from method.ours.generation import (
    generate_constraints_for_input_groups,
    generate_value_for_input_group,
    generate_values_for_input_groups,
    fill_form_with_value_table,
    submit_form
)
from method.ours.feedback import (
    get_local_feedback,
    get_global_feedback
)

In [3]:
load_dotenv()
openai.api_key = os.getenv("OPENAI_API_KEY")

# Global Variables
HEADLESS = False
TEXT_EMBEDDING_METHOD = 'ADA' # ['ADA', 'WORD2VEC', 'SPACY']
GRAPH_EMBEDDING_METHOD = 'NODE2VEC' # ['NODE2VEC', 'GCN']

URL = 'https://www.aircanada.com/ca/en/aco/home.html'

In [4]:
driver = create_driver(HEADLESS)
driver.get(URL)

In [32]:
form = driver.find_elements(By.TAG_NAME, 'form')[1]

In [6]:
history_table = HistoryTable(
    url=URL,
    xpath=get_xpath(driver, form)
)

In [7]:
for element in form.find_elements(By.TAG_NAME, 'input'):
    try:
        element.clear()
    except:
        pass

# Processing

In [8]:
html = driver.find_element(By.TAG_NAME, 'body').get_attribute('outerHTML')
relation_graph, input_groups = parse_form(
    driver,
    form,
    TEXT_EMBEDDING_METHOD=TEXT_EMBEDDING_METHOD
)

Computing transition probabilities:   0%|          | 0/340 [00:00<?, ?it/s]

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 34/34 [00:10<00:00,  3.22it/s]
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/openai/embeddings_utils.py:68: RuntimeWarning: invalid value encountered in scalar divide
  return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))


# Generation

In [9]:
value_table = generate_constraints_for_input_groups(input_groups)

  Prompt: [
  {
    "role": "system",
    "content": "Instructions:\nYour task is to generate a set of constraints for web form fields. Your decisions must be made independently without seeking user assistance or additional information. If there are multiple ways to express constraints, use the least number of constraints to describe them. Only generate the constraints and refrain from explaining your answers. Only generate constraints for the input field in question, not those in the relevant information section. You must choose your constraints in the format of our modified version of the Jest library in JavaScript. The list of functions in this modified format are:\n1. toBeEqual(value) # the input field value is exactly equal to the given value\n2. toHaveLengthCondition(condition, value) # the length of the input field value matches the given condition\n3. toBeTruthy() # the input field value is truthy and not empty (not false, 0, '', null, undefined, or NaN)\n4. toHaveCompareCondit

  Response: {
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "message": {
        "content": "expect(field('bkmgFlights_tripTypeSelector_M'))\n.toBeEqual('M')",
        "role": "assistant"
      }
    }
  ],
  "created": 1689278065,
  "id": "chatcmpl-7bwgLkBXeegOJ7iGfy6WXHb1ZmAf7",
  "model": "gpt-4-0613",
  "object": "chat.completion",
  "usage": {
    "completion_tokens": 17,
    "prompt_tokens": 665,
    "total_tokens": 682
  }
}
  Total running cost: $0.060
  Prompt: [
  {
    "role": "system",
    "content": "Instructions:\nYour task is to generate a set of constraints for web form fields. Your decisions must be made independently without seeking user assistance or additional information. If there are multiple ways to express constraints, use the least number of constraints to describe them. Only generate the constraints and refrain from explaining your answers. Only generate constraints for the input field in question, not those in the relevant inform

  Response: {
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "message": {
        "content": "expect(field('bkmgFlights_destination_trip_1'))\n.toBeTruthy()\n.toBeAlphabetical()\n.not.toEqual(field('bkmgFlights_origin_trip_1'))",
        "role": "assistant"
      }
    }
  ],
  "created": 1689278074,
  "id": "chatcmpl-7bwgU96oKdEtxC0hD6RA4Ku29X9PD",
  "model": "gpt-4-0613",
  "object": "chat.completion",
  "usage": {
    "completion_tokens": 34,
    "prompt_tokens": 643,
    "total_tokens": 677
  }
}
  Total running cost: $0.120
  Prompt: [
  {
    "role": "system",
    "content": "Instructions:\nYour task is to generate a set of constraints for web form fields. Your decisions must be made independently without seeking user assistance or additional information. If there are multiple ways to express constraints, use the least number of constraints to describe them. Only generate the constraints and refrain from explaining your answers. Only generate constrai

In [10]:
value_table = generate_values_for_input_groups(input_groups, value_table)

  Prompt: [
  {
    "role": "system",
    "content": "Your task is to generate a value for a web form field based on the form field information and a set of constraints on the field. Your decisions must be made independently without seeking user assistance or additional information. For each user prompt, you need to generate one value that satisfies the constraints while keeping in mind the nature of the input from the available information. Only generate value and refrain from explaining your answers. Only generate value for the input field in question, and not the ones in the relevant information section."
  },
  {
    "role": "user",
    "content": "The following are all the labels in the form, which provide context for the functionality of the form:  Round-trip ,  One-way ,  Multi-city/Stopover ,  Book with points , From, To, Departure, Return\n    We are generating filling values for the following input field:\n    input: <input id=\"bkmgFlights_tripTypeSelector_R\" name=\"tripTyp

  Response: {
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "message": {
        "content": "Toronto",
        "role": "assistant"
      }
    }
  ],
  "created": 1689278107,
  "id": "chatcmpl-7bwh1dGoIgqpU4lveqliYxoI0dLih",
  "model": "gpt-4-0613",
  "object": "chat.completion",
  "usage": {
    "completion_tokens": 1,
    "prompt_tokens": 290,
    "total_tokens": 291
  }
}
  Total running cost: $0.210
  Prompt: [
  {
    "role": "system",
    "content": "Your task is to generate a value for a web form field based on the form field information and a set of constraints on the field. Your decisions must be made independently without seeking user assistance or additional information. For each user prompt, you need to generate one value that satisfies the constraints while keeping in mind the nature of the input from the available information. Only generate value and refrain from explaining your answers. Only generate value for the input field in question, an

In [45]:
fill_form_with_value_table(driver, value_table, input_groups)

<input>value is R</input> at y: (567, 597), x: (219, 319)
R
<input>value is O</input> at y: (567, 597), x: (334, 421)
O
<input>value is M</input> at y: (567, 597), x: (437, 591)
M
<input></input> at y: (574, 594), x: (652, 905)
true
<input></input> at y: (628, 678), x: (198, 466)
Vancouver
<input></input> at y: (628, 678), x: (536, 804)
Toronto
<input>DD/MM</input> at y: (628, 678), x: (833, 958)
15/07
<input>DD/MM</input> at y: (628, 678), x: (978, 1093)
20/07


In [47]:
submit_form(driver, input_groups)

In [13]:
new_html = driver.find_element(By.TAG_NAME, 'body').get_attribute('outerHTML')
global_feedback = get_global_feedback(html, new_html, remove_form_children=False)

In [14]:
history_table.add(
    value_table.get_values_dict(),
    'base',
    global_feedback,
    driver.current_url
)

/Users/parsaalian/Desktop/Research/webform testing project/webform-testing-code/implementation/method/ours/history.py:83: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.table = self.table.append({


# Feedback

In [21]:
new_form = driver.find_elements(By.TAG_NAME, 'form')[1]

In [22]:
new_relation_graph, new_input_groups = parse_form(
    driver,
    new_form,
    prev_relation_graph=relation_graph,
    TEXT_EMBEDDING_METHOD=TEXT_EMBEDDING_METHOD
)

Computing transition probabilities:   0%|          | 0/353 [00:00<?, ?it/s]

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 37/37 [00:10<00:00,  3.62it/s]
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/openai/embeddings_utils.py:68: RuntimeWarning: invalid value encountered in scalar divide
  return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))


In [23]:
global_feedback = get_global_feedback(html, new_html, remove_form_children=True)

In [24]:
value_table = generate_constraints_for_input_groups(
    new_input_groups,
    value_table,
    global_feedback
)

  Prompt: [
  {
    "role": "system",
    "content": "Instructions:\nYour task is to generate a set of constraints for web form fields. Your decisions must be made independently without seeking user assistance or additional information. If there are multiple ways to express constraints, use the least number of constraints to describe them. Only generate the constraints and refrain from explaining your answers. Only generate constraints for the input field in question, not those in the relevant information section. You must choose your constraints in the format of our modified version of the Jest library in JavaScript. The list of functions in this modified format are:\n1. toBeEqual(value) # the input field value is exactly equal to the given value\n2. toHaveLengthCondition(condition, value) # the length of the input field value matches the given condition\n3. toBeTruthy() # the input field value is truthy and not empty (not false, 0, '', null, undefined, or NaN)\n4. toHaveCompareCondit

In [25]:
value_table = generate_values_for_input_groups(new_input_groups, value_table)

  Prompt: [
  {
    "role": "system",
    "content": "Your task is to generate a value for a web form field based on the form field information and a set of constraints on the field. Your decisions must be made independently without seeking user assistance or additional information. For each user prompt, you need to generate one value that satisfies the constraints while keeping in mind the nature of the input from the available information. Only generate value and refrain from explaining your answers. Only generate value for the input field in question, and not the ones in the relevant information section."
  },
  {
    "role": "user",
    "content": "The following are all the labels in the form, which provide context for the functionality of the form:  Round-trip ,  One-way ,  Multi-city/Stopover ,  Book with points , From, To, Departure, Return\n    We are generating filling values for the following input field:\n    input: <input id=\"bkmgFlights_tripTypeSelector_R\" name=\"tripTyp

  Response: {
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "message": {
        "content": "New York",
        "role": "assistant"
      }
    }
  ],
  "created": 1689275951,
  "id": "chatcmpl-7bw8FEJ5ia1DTQ7kpZXLuVU7rMw9w",
  "model": "gpt-4-0613",
  "object": "chat.completion",
  "usage": {
    "completion_tokens": 2,
    "prompt_tokens": 313,
    "total_tokens": 315
  }
}
  Total running cost: $0.336
  Prompt: [
  {
    "role": "system",
    "content": "Your task is to generate a value for a web form field based on the form field information and a set of constraints on the field. Your decisions must be made independently without seeking user assistance or additional information. For each user prompt, you need to generate one value that satisfies the constraints while keeping in mind the nature of the input from the available information. Only generate value and refrain from explaining your answers. Only generate value for the input field in question, a

In [26]:
fill_form_with_value_table(driver, value_table, new_input_groups)

<input>value is R</input> at y: (567, 597), x: (219, 319)
R
<input>value is O</input> at y: (567, 597), x: (334, 421)
O
<input>value is M</input> at y: (567, 597), x: (437, 591)
M
<input></input> at y: (574, 594), x: (652, 905)
true
<input></input> at y: (628, 678), x: (198, 466)
London
<input></input> at y: (628, 678), x: (536, 804)
New York
<input>DD/MM</input> at y: (628, 678), x: (833, 958)
15/07
<input>DD/MM</input> at y: (628, 678), x: (978, 1093)
20/07


In [27]:
submit_form(driver, new_input_groups)

In [28]:
new_html = driver.find_element(By.TAG_NAME, 'body').get_attribute('outerHTML')
global_feedback = get_global_feedback(html, new_html, remove_form_children=False)

history_table.add(
    value_table.get_values_dict(),
    'base',
    global_feedback,
    driver.current_url
)

/Users/parsaalian/Desktop/Research/webform testing project/webform-testing-code/implementation/method/ours/history.py:83: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.table = self.table.append({


# Constraint Negation

In [15]:
form_context = get_form_context(input_groups)

In [17]:
get_attr = lambda x, a, d: x[a] if a in x else d

for field_id, entry in value_table.entries.items():
    input_type = get_attr(entry.input_group.node.element.attrs, 'type', 'text')
    if input_type in ['radio', 'checkbox']:
        continue
    
    for i in range(len(entry.constraints)):
        value_table_copy = value_table.copy()
    
        value_table_copy.entries[field_id].constraints = [value_table.entries[field_id].constraints[i].copy()]
        value_table_copy.entries[field_id].constraints[0].flip_negative()


        input_group = value_table_copy.entries[field_id].input_group

        value_table_copy = generate_value_for_input_group(
            input_group,
            value_table_copy,
            form_context=form_context
        )

        fill_form_with_value_table(driver, value_table_copy, input_groups)

        time.sleep(2)

        submit_form(driver, input_groups)
        
        time.sleep(2)
        
        new_html = driver.find_element(By.TAG_NAME, 'body').get_attribute('outerHTML')
        global_feedback = get_global_feedback(html, new_html, remove_form_children=False)

        history_table.add(
            value_table_copy.get_values_dict(),
            str(type(value_table_copy.entries[field_id].constraints[0])),
            global_feedback,
            driver.current_url
        )
        
        if len(global_feedback) == 0:
            value_table.entries[field_id].constraints[i].approve()

  Prompt: [
  {
    "role": "system",
    "content": "Your task is to generate a value for a web form field based on the form field information and a set of constraints on the field. Your decisions must be made independently without seeking user assistance or additional information. For each user prompt, you need to generate one value that satisfies the constraints while keeping in mind the nature of the input from the available information. Only generate value and refrain from explaining your answers. Only generate value for the input field in question, and not the ones in the relevant information section."
  },
  {
    "role": "user",
    "content": "The following are all the labels in the form, which provide context for the functionality of the form:  Round-trip ,  One-way ,  Multi-city/Stopover ,  Book with points , From, To, Departure, Return\n    We are generating filling values for the following input field:\n    input: <input id=\"bkmgFlights_origin_trip_1\" name=\"bkmgFlights_

/Users/parsaalian/Desktop/Research/webform testing project/webform-testing-code/implementation/method/ours/history.py:83: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.table = self.table.append({


  Prompt: [
  {
    "role": "system",
    "content": "Your task is to generate a value for a web form field based on the form field information and a set of constraints on the field. Your decisions must be made independently without seeking user assistance or additional information. For each user prompt, you need to generate one value that satisfies the constraints while keeping in mind the nature of the input from the available information. Only generate value and refrain from explaining your answers. Only generate value for the input field in question, and not the ones in the relevant information section."
  },
  {
    "role": "user",
    "content": "The following are all the labels in the form, which provide context for the functionality of the form:  Round-trip ,  One-way ,  Multi-city/Stopover ,  Book with points , From, To, Departure, Return\n    We are generating filling values for the following input field:\n    input: <input id=\"bkmgFlights_origin_trip_1\" name=\"bkmgFlights_

/Users/parsaalian/Desktop/Research/webform testing project/webform-testing-code/implementation/method/ours/history.py:83: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.table = self.table.append({


  Prompt: [
  {
    "role": "system",
    "content": "Your task is to generate a value for a web form field based on the form field information and a set of constraints on the field. Your decisions must be made independently without seeking user assistance or additional information. For each user prompt, you need to generate one value that satisfies the constraints while keeping in mind the nature of the input from the available information. Only generate value and refrain from explaining your answers. Only generate value for the input field in question, and not the ones in the relevant information section."
  },
  {
    "role": "user",
    "content": "The following are all the labels in the form, which provide context for the functionality of the form:  Round-trip ,  One-way ,  Multi-city/Stopover ,  Book with points , From, To, Departure, Return\n    We are generating filling values for the following input field:\n    input: <input id=\"bkmgFlights_origin_trip_1\" name=\"bkmgFlights_

...

/Users/parsaalian/Desktop/Research/webform testing project/webform-testing-code/implementation/method/ours/history.py:83: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.table = self.table.append({


  Prompt: [
  {
    "role": "system",
    "content": "Your task is to generate a value for a web form field based on the form field information and a set of constraints on the field. Your decisions must be made independently without seeking user assistance or additional information. For each user prompt, you need to generate one value that satisfies the constraints while keeping in mind the nature of the input from the available information. Only generate value and refrain from explaining your answers. Only generate value for the input field in question, and not the ones in the relevant information section."
  },
  {
    "role": "user",
    "content": "The following are all the labels in the form, which provide context for the functionality of the form:  Round-trip ,  One-way ,  Multi-city/Stopover ,  Book with points , From, To, Departure, Return\n    We are generating filling values for the following input field:\n    input: <input id=\"bkmgFlights_travelDates_1-formfield-2\" name=\"

/Users/parsaalian/Desktop/Research/webform testing project/webform-testing-code/implementation/method/ours/history.py:83: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.table = self.table.append({


  Prompt: [
  {
    "role": "system",
    "content": "Your task is to generate a value for a web form field based on the form field information and a set of constraints on the field. Your decisions must be made independently without seeking user assistance or additional information. For each user prompt, you need to generate one value that satisfies the constraints while keeping in mind the nature of the input from the available information. Only generate value and refrain from explaining your answers. Only generate value for the input field in question, and not the ones in the relevant information section."
  },
  {
    "role": "user",
    "content": "The following are all the labels in the form, which provide context for the functionality of the form:  Round-trip ,  One-way ,  Multi-city/Stopover ,  Book with points , From, To, Departure, Return\n    We are generating filling values for the following input field:\n    input: <input id=\"bkmgFlights_travelDates_1-formfield-2\" name=\"

/Users/parsaalian/Desktop/Research/webform testing project/webform-testing-code/implementation/method/ours/history.py:83: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.table = self.table.append({


In [18]:
history_table.to_test_case('output.py')